In [4]:
# Read in Libraries
from __future__ import division, print_function
from logbook import Logger, StreamHandler
import sys
StreamHandler(sys.stdout).push_application()
log = Logger('Logbook')

import utils; reload(utils)
from utils import *
import gc
# from __future__ import division, print_function
from theano.sandbox import cuda
from vgg16bn import Vgg16BN
from sklearn import metrics

def accuracyfunc(y_act, y_pred):
    return metrics.accuracy_score(np.argmax(y_act, axis=1), np.argmax(y_pred, axis=1))

# Set Parameters and check files
input_exists = True
log.info('Set Paramters')
path = "../data/fish/"
batch_size=64

# Read in our VGG pretrained model
log.info('Get VGG')
model = vgg_ft_bn(8)

# Create our VGG model
log.info('Create VGG')
vgg640 = Vgg16BN((360, 640)).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

# get labels
(val_classes, trn_classes, val_labels, trn_labels,
    val_filenames, filenames, test_filenames) = get_classes(path)

# Read in filenames
log.info('Read filenames')
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

[2017-02-06 20:45:01.128384] INFO: Logbook: Set Paramters
[2017-02-06 20:45:01.129851] INFO: Logbook: Get VGG
[2017-02-06 20:45:04.556287] INFO: Logbook: Create VGG
Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.
[2017-02-06 20:45:05.077589] INFO: Logbook: Read filenames


In [5]:
log.info('Read in data')
if not input_exists:

    batches = get_batches(path+'train', batch_size=batch_size)
    val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
    (val_classes, trn_classes, val_labels, trn_labels, 
        val_filenames, filenames, test_filenames) = get_classes(path)
    
    
    # Fetch our large images 
    log.info('Fetch images')
    trn = get_data(path+'train', (360,640))
    val = get_data(path+'valid', (360,640))
    test = get_data(path+'test', (360,640))
    
    # Precompute the output of the convolutional part of VGG
    log.info('Get VGG output')
    conv_val_feat = vgg640.predict(val, batch_size=32, verbose=1)
    conv_trn_feat = vgg640.predict(trn, batch_size=32, verbose=1)
    conv_test_feat = vgg640.predict(test, batch_size=32, verbose=1)
    log.info('Write VGG output')
    save_array(path+'results/conv_val_feat.dat', conv_val_feat)
    save_array(path+'results/conv_trn_feat.dat', conv_trn_feat) 
    save_array(path+'results/conv_test_feat.dat', conv_test_feat)     

    # For memory purposes delete out the original train and validation
    log.info('Clear up memory')
    del trn, val, test
    gc.collect()

conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_trn_feat = load_array(path+'results/conv_trn_feat.dat') 
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

[2017-02-06 20:45:05.103309] INFO: Logbook: Read in data


In [80]:
# Our Convolutional Net Architecture
log.info('Create and fit CNN')
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

# Set up the fully convolutional net (FCN); 
conv_layers,_ = split_at(vgg640, Convolution2D)
nf=128; p=0. # No dropout

lrg_model = []
predsls = []
pvalsls = []
bags = 6

[2017-02-06 22:12:27.292619] INFO: Logbook: Create and fit CNN


In [81]:
for i in range(bags):
    log.info('Train round' + str(i))
    lrg_model.append(Sequential(get_lrg_layers()))
    # lrg_model.summary()
    lrg_model[i].compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    lrg_model[i].fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
                 validation_data=(conv_val_feat, val_labels))
    lrg_model[i].optimizer.lr=1e-5
    lrg_model[i].fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=6,
                 validation_data=(conv_val_feat, val_labels))

    ## Evaluate the model
    #log.info('Evaluate')
    #lrg_model[i].evaluate(conv_val_feat, val_labels)

    # Make our prediction on the lrg_model layer
    log.info('Output Prediction')
    predsls.append(lrg_model[i].predict(conv_test_feat, batch_size=batch_size)) # or try 32 batch_size
    pvalsls.append(lrg_model[i].predict(conv_val_feat, batch_size=batch_size))
    val_score = "%.3f" % metrics.log_loss(val_labels, sum(pvalsls)/len(pvalsls))
    acc_score = "%.3f" % accuracyfunc(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .99))
    log.info('Bagged Validation Logloss ' + str(val_score))
    log.info('Bagged Validation Logloss ' + str(acc_score))

[2017-02-06 22:12:29.630698] INFO: Logbook: Train round0
th
Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 16s - loss: 0.6650 - acc: 0.7882 - val_loss: 1.1273 - val_acc: 0.6920
Epoch 2/2
3277/3277 [==============================] - 16s - loss: 0.1135 - acc: 0.9689 - val_loss: 0.3739 - val_acc: 0.9220
Train on 3277 samples, validate on 500 samples
Epoch 1/6
3277/3277 [==============================] - 16s - loss: 0.0348 - acc: 0.9927 - val_loss: 0.2370 - val_acc: 0.9600
Epoch 2/6
3277/3277 [==============================] - 16s - loss: 0.0090 - acc: 0.9994 - val_loss: 0.1620 - val_acc: 0.9660
Epoch 3/6
3277/3277 [==============================] - 16s - loss: 0.0196 - acc: 0.9963 - val_loss: 0.2010 - val_acc: 0.9520
Epoch 4/6
3277/3277 [==============================] - 16s - loss: 0.0068 - acc: 0.9991 - val_loss: 0.2157 - val_acc: 0.9420
Epoch 5/6
3277/3277 [==============================] - 16s - loss: 0.0041 - acc: 0.9991 - val_lo

KeyboardInterrupt: 

In [82]:
metrics.log_loss(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .99))

0.18894760163128377

In [62]:
preds = sum(predsls)/len(predsls)
subm = do_clip(preds,.99)
subm_name = path+'results/subm_bb_conv_lrg0206C.csv.gz'
pred_name = path+'results/pred_bb_conv_lrg0206C.csv.gz'

classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.to_csv(subm_name, index=False, compression='gzip')
subm1 = pd.DataFrame(preds, columns=classes)
subm1.insert(0, 'image', raw_test_filenames)
subm1.to_csv(pred_name, index=False, compression='gzip')

In [61]:
FileLink(subm_name)

/home/ubuntu/fish/data/fish/results/subm_bb_conv_lrg0206C.csv.gz

In [84]:
gen = image.ImageDataGenerator(horizontal_flip=True)
gen

In [ ]:
input_exists =False
if not input_exists:

    batches = get_batches(path+'train',gen, batch_size=batch_size)
    batchestmp = get_batches(path+'train', batch_size=batch_size)
    val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
    (val_classes, trn_classes, val_labels, trn_labels, 
        val_filenames, filenames, test_filenames) = get_classes(path)
    
    
    # Fetch our large images 
    log.info('Fetch images')
    trn = get_data(path+'train', (360,640))
    val = get_data(path+'valid', (360,640))
    test = get_data(path+'test', (360,640))
    
    # Precompute the output of the convolutional part of VGG
    log.info('Get VGG output')
    conv_val_feat = vgg640.predict(val, batch_size=32, verbose=1)
    conv_trn_feat = vgg640.predict(trn, batch_size=32, verbose=1)
    conv_test_feat = vgg640.predict(test, batch_size=32, verbose=1)
    log.info('Write VGG output')
    save_array(path+'results/conv_val_feat.dat', conv_val_feat)
    save_array(path+'results/conv_trn_feat.dat', conv_trn_feat) 
    save_array(path+'results/conv_test_feat.dat', conv_test_feat)     

    # For memory purposes delete out the original train and validation
    log.info('Clear up memory')
    del trn, val, test
    gc.collect()



Found 3277 images belonging to 8 classes.
Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.
[2017-02-06 22:43:14.473802] INFO: Logbook: Fetch images
Found 3277 images belonging to 8 classes.
